In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from config.path import PATH # config/path.py to manage your dataset paths
coco_path = PATH["COCO"]

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e:
    # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
    print(e)

import libraries

In [ ]:
from data.ssd import SSDDataset
from models.SSD import SSD300
from models.RetinaNet import RetinaNet
from utils.visualize import visualize_detections

### Load COCO dataset

In [ ]:
image_path = f"{coco_path}/images/train2017"
annotation_path = f"{coco_path}/annotations/large_pedestrian_train.json"
train_dataset = SSDDataset(image_path, annotation_path, num_examples=50, shuffle=False)

Visualize a single data sample.

In [ ]:
image, boxes, classes = train_dataset.get_item(train_dataset.image_ids[0])
visualize_detections(image, boxes, classes, tf.ones_like(classes))

### Load your model

In [ ]:
# SSD300
num_classes = train_dataset.num_classes()
model = SSD300(num_classes)

### Load data pipeline

Each sample which consist of (image, bounding boxes, classes), will be converted into image, ground truths of anchors (N, anchor, 5). 5 is [dx,dy,dw,dh,class_id]

In [ ]:
train_tfds = train_dataset.load_tfds(batch_size=1, model=model)

In [ ]:
items = next(iter(train_tfds))
for item in items:
    print(item.shape)

### training

In [ ]:
optimizer = tf.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer)
model.fit(train_tfds, epochs=5, verbose=1,)

### load a pretrained model and visualize inference

In [ ]:
weights_dir = "experiments/large_Pedestrian_SSD/20220714-171249/"
latest_checkpoint = tf.train.latest_checkpoint(weights_dir)
model.load_weights(latest_checkpoint)

In [ ]:
images = items[0]
print(images.shape)

In [ ]:
inference_model = model.inference(confidence_threshold=0.3)
detections = inference_model(images)

In [ ]:
from utils.visualize import visualize_inference
visualize_inference(images[0], train_dataset.labels, detections)